In [ ]:
# show conda environment
import sys
print(sys.executable)

In [ ]:
import os
import sp_peaks
from sp_peaks import slics
from sp_peaks import mapping
from sp_peaks import summary_statistics
from sp_peaks import plotting
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from lenspack.geometry.projections.gnom import radec2xy
from lenspack.utils import bin2d
from lenspack.image.inversion import ks93
import lenspack.peaks as peaks
from lenspack.starlet_l1norm import noise_coeff, get_l1norm_noisy
from lenspack.image.transforms import starlet2d
from astropy.stats import mad_std

In [ ]:
# Define root directory where the simulation files are located
root_directory = "/n17data/tersenov/SLICS/Cosmo_DES"

# Directories to exclude
exclude_dirs = ["SLICS_HR", "fid_a", "fid_f"]

# Open the master text file in write mode
master_file_path = ".././input/master_file.txt"
with open(master_file_path, "w") as master_file:
    # Iterate over the files and subdirectories in the root directory
    for root, dirs, files in os.walk(root_directory):
        # Exclude the specified directories
        dirs[:] = [d for d in dirs if d not in exclude_dirs]
        
        for file_name in files:
            # Write the file path to the master file
            file_path = os.path.join(root, file_name)
            master_file.write(file_path + "\n")

In [ ]:
# Read the file paths from master_file.txt
filename = ".././input/master_file.txt"
with open(filename, 'r') as file:
    file_paths = file.readlines()
    file_paths = [path.strip() for path in file_paths]

filename_data = slics.parse_SLICS_filenames(file_paths)

In [ ]:
# Path to the .dat file
dat_file_path = "/home/tersenov/shear-pipe-peaks/example/CosmoTable.dat"

# Read the cosmological parameters from the .dat file
cosmo_params = slics.read_SLICS_cosmo_params(dat_file_path)

In [ ]:
dat_file_path = "/home/tersenov/shear-pipe-peaks/example/CosmoTable.dat"
mapped_params = slics.map_cosmo_params_to_data(filename_data, dat_file_path)

# Generate the mass map from the catalog, make the S/N maps and count the peaks

Catalog columns:

0. RA
1. DEC
2. e1_data
3. e2_data
4. w 
5. redshift_true_sim
6. gamma1_sim
7. gamma2_sim
8. kappa_sim
9. s_metacal_data

In [ ]:
# Constants and Parameters
CATALOG_FILE = "/n17data/tersenov/SLICS/Cosmo_DES/16_a/LOS4/DES_MocksCat_16_a_4_Bin3_LOS4_R4.dat"
N_GAL = 7
PIX_ARCMIN = 0.4
SHAPE_NOISE = 0.44
NSCALES = 5
NBINS = 40 
KAPPA_SNR = np.linspace(-2, 6, 31)

Loading catalog

In [ ]:
# Load the catalog data 
catalog_data = slics.read_catalogue(CATALOG_FILE)

In [ ]:
ra = catalog_data['RA']
dec = catalog_data['Dec']
g1_sim = catalog_data['gamma1_sim']
g2_sim = catalog_data['gamma2_sim']
kappa_sim = catalog_data['kappa_sim']

## Making the galaxy number density map

Compute number of pixels, Switch from RA, DEC to pixels

In [ ]:
# Define the size of the pixel grid in degrees and the pixel size in arcminutes
size_x_deg = 10
size_y_deg = 10
pixel_size_emap_amin=0.4 # pixel size in arcminutes --> will give us the pixel size that we want our mass map to be

# Calculate the number of pixels in the x and y directions
Nx = int(size_x_deg / pixel_size_emap_amin * 60)
Ny = int(size_y_deg / pixel_size_emap_amin * 60)

# Project (ra,dec) -> (x,y)
# The radec2xy() function takes the mean RA and DEC values of the galaxies as well as the RA and DEC values of each individual galaxy as input
x, y = radec2xy(np.mean(ra), np.mean(dec), ra, dec)

In [ ]:
# create the galaxy number density map
galmap = bin2d(x, y, npix=(Nx,Ny))

In [ ]:
plt.imshow(galmap, cmap='inferno', interpolation='nearest', origin='lower')
plt.colorbar()

In [ ]:
# smooth the galaxy map with a Gaussian filter of width 2 arcmin
smoothed_galmap = mapping.smooth_kappa_map(galmap, pixel_size_emap_amin)

In [ ]:
plt.imshow(smoothed_galmap, cmap='inferno', interpolation='nearest', origin='lower', vmin=0, vmax=2)
plt.colorbar()

Create kappa map and make an SNR map from it

In [ ]:
# Bin the shear field
e1map, e2map = mapping.bin_shear_field(ra, dec, g1_sim, g2_sim)

# Create the kappa map
kappaE, _ = ks93(e1map, -e2map)

# Add noise to the kappa map
kappaE_noisy, noise_map_CFIS_z05 = mapping.add_noise_to_kappa_map(kappaE, SHAPE_NOISE, N_GAL, PIX_ARCMIN)

# Smooth the noisy kappa map
kappaE_noisy_smoothed = mapping.smooth_kappa_map(kappaE_noisy, PIX_ARCMIN) 

# Compute SNR map
snr = mapping.convert_to_snr_map(kappaE_noisy_smoothed, kappaE_noisy_smoothed)

In [ ]:
# function to add shape noise that depends on the galaxy number density

def add_varying_noise_to_kappa_map(kappa_map, gal_density_map, pix_arcmin, shape_noise, density_threshold=0.5):
    # Calculate the mean galaxy number density
    mean_density = np.mean(gal_density_map)
    
    # Create a mask for pixels with galaxy densities above the threshold
    valid_mask = gal_density_map >= (density_threshold * mean_density)

    # Calculate the shape noise level based on the galaxy number density
    sigma_noise_CFIS = shape_noise / (np.sqrt(2 * gal_density_map * pix_arcmin**2))
    
    # Generate a noise map with the same shape as the input kappa map
    noise_map_CFIS_z05 = sigma_noise_CFIS * np.random.randn(*kappa_map.shape)

    # Apply the mask to the noise map to set pixels below the threshold to zero noise
    noise_map_CFIS_z05[~valid_mask] = 0
    
    # Add the noise map to the kappa map
    kappa_map_noisy = kappa_map + noise_map_CFIS_z05
    
    return kappa_map_noisy, noise_map_CFIS_z05

# Add noise to the kappa map
var_kappaE_noisy, var_noise_map_CFIS_z05 = add_varying_noise_to_kappa_map(kappaE, galmap, PIX_ARCMIN, SHAPE_NOISE)

In [ ]:
plotting.plot_map(kappaE, title="Kappa Map", vmin=-0.01, vmax=0.01)

In [ ]:
plotting.plot_map(var_kappaE_noisy, vmin=-0.1, vmax=0.1, title='Noisy kappa map')

In [ ]:
# smooth the noisy kappa map
var_kappaE_noisy_smoothed = mapping.smooth_kappa_map(var_kappaE_noisy, PIX_ARCMIN) 

In [ ]:
plotting.plot_map(var_kappaE_noisy_smoothed, vmin=-0.1, vmax=0.1, title='Smoothed noisy kappa map')

Peak counts on the SNR map

In [ ]:
# Compute peak counts 
kappa_th_center_snr, peak_counts_single = summary_statistics.compute_single_scale_peak_counts(snr, KAPPA_SNR)

Multiscale SNR maps

In [ ]:
# Compute multiscale SNR maps
multiscale_snr_maps = mapping.compute_multiscale_snr_maps(kappaE, noise_map_CFIS_z05, NSCALES)

In [ ]:
# Compute peak counts for each scale
kappa_th_center_snr, peak_counts_multi = summary_statistics.compute_multiscale_peak_counts(multiscale_snr_maps, KAPPA_SNR)

$\ell_1$-norm

In [ ]:
# Call the function to compute the L1-norm histogram
bins_l1, l1norm_histogram = get_l1norm_noisy(kappaE, noise_map_CFIS_z05, NSCALES, NBINS*2)

# Plotting

Plot:
1. Noiseless mass map
2. Noisy mass map
3. Noisy smoothed mass map
4. SNR map

In [ ]:
# Define the map data and titles
map_data = [kappaE, kappaE_noisy, kappaE_noisy_smoothed, snr]
titles = ['Noiseless Mass Map', 'Noisy Mass Map', 'Noisy Smoothed Mass Map', 'SNR Map']

# Define the vmin and vmax values for each map (adjust these values as needed)
vmin_values = [-0.01, -0.1, -0.1, None]
vmax_values = [0.01, 0.1, 0.1, None]

# Plot each map using the plot_map function
for data, title, vmin, vmax in zip(map_data, titles, vmin_values, vmax_values):
    plotting.plot_map(data, title=title, vmin=vmin, vmax=vmax)

Plot the SNR maps of the 5 different scales and the coarse map

In [ ]:
# Define vmin and vmax values for each SNR map
vmin_values = [-0.5, -3, -4, -4, -4, -4]  # Replace with your desired vmin values
vmax_values = [0.5, 3, 4, 4, 4, 4]  # Replace with your desired vmax values

scales_arcmin = [2**(i+1) * PIX_ARCMIN for i in range(len(multiscale_snr_maps))]

for i, (snr_map, scale_arcmin) in enumerate(zip(multiscale_snr_maps, scales_arcmin)):
    title = f"SNR Map (Scale {scale_arcmin:.1f} arcmin)"
    plotting.plot_map(snr_map, title=title, cmap='inferno', vmin=vmin_values[i], vmax=vmax_values[i])

In [ ]:
# Plot single-scale peak count histogram
plotting.plot_peak_count_histograms(kappa_th_center_snr, peak_counts_single, 
                           'Peak Counts Histogram for Gaussian SNR Map', 'SNR', 'Peak Counts')

In [ ]:
# Plot multiscale peak count histograms
plotting.plot_peak_count_histograms(kappa_th_center_snr, peak_counts_multi, 
                           'Peak Counts for Different Scales', 'SNR', 'Peak Counts', log_scale=False)

In [ ]:
# Plot multiscale peak count histograms
plotting.plot_peak_count_histograms(kappa_th_center_snr, peak_counts_multi, 
                           'Peak Counts for Different Scales', 'SNR', 'Peak Counts', log_scale=True)

In [ ]:
# Plot l1-norm histograms for different scales
plotting.plot_l1norm_histograms(bins_l1, l1norm_histogram, 'L1-norm Histograms for Different Scales', 'SNR', 'L1-norm', log_scale=False)